This Notebook contains the code used for training and saving the trained model along with testing the model for batch of validation data points. 
You are given an untrained model file with filename as "untrained_model.pt" and a csv file (val_norm.csv) containing the validation data points.

Your task is to deploy an API for getting inference from the model file and using the API generate predictions for the validation data points. You may use Postman for hitting the API. You can create API using framework of your choice. 

You should be able to deploy the model inference API in some free cloud VM like from Heroku or AWS free tier. If you can not deploy on cloud then at least you will have to deploy in local environment but for this role you need to deploy models in cloud as API. Passing one row of val_norm.csv as parameter to API to generate predictions is sufficient and save the predictions in a text file. 

You will need to submit the text file with the predictions, input row of val_norm.csv as well as the code for the API. If you have deployed it in the cloud, submit the URL as well. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

In [ ]:
class PoseDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with pose landmark locations.

        """
        self.pose_data = pd.read_csv(csv_file)


    def __len__(self):
        return len(self.pose_data)

    def __getitem__(self, idx):
        
        item = torch.tensor(self.pose_data.iloc[idx, :-1], dtype=torch.float32)
        label = torch.tensor(self.pose_data.iloc[idx, -1], dtype=torch.int64)
        return item, label

In [ ]:
train_data = PoseDataset("train_norm.csv")
test_data = PoseDataset("test_norm.csv")
val_data = PoseDataset("val_norm.csv")

In [ ]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=0)

In [ ]:
class DNN(nn.Module):

    def __init__(self):
        super(DNN, self).__init__()
        # We will create a simple neural network with 2 fully connected layers for our use case
        self.fc1 = nn.Linear(99, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.drop1 = nn.Dropout(p=0.4)
        self.fc4 = nn.Linear(128, 28) # 8 classes

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.drop1(x)
        x = self.fc4(x)
        return x


dnn = DNN()
# torchsummary.summary(dnn, (99,), 16)

In [ ]:
import torch.optim as optim

best_val_loss = 99999
best_model = None
best_val_accuracy = 0
# Create a function to train and validate the model and print out the accuracy and loss after every epoch.
def train_and_validate(model, criterion, optimizer, train_loader, validation_loader):
    """Train and validate the model on training and validation datasets

    Args:
        model (nn.Module): Neural network to be trained
        criterion (Callable): Loss function (Cross Entropy Loss)
        optimizer (Callable): Optimizer to use for training the model (SGD)
        train_loader (Generator): Train data loader
        validation_loader (Generator): Validation data loader
        BATCH_SIZE (int): Batch size to be used for training and validation

    Returns:
        Tuple: Tuple of best model and best model validation accuracy
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print('Epoch:', epoch + 1)
    print('Training...')
    model.train()
    train_loss, train_accuracy = 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
         # print(output)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        # print(pred)
        train_accuracy += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_data)
    train_accuracy /= len(train_data)
    print('Train loss: {:.4f}, Train accuracy: {:.4f}'.format(train_loss, train_accuracy))
    print('Validation...')
    model.eval()
    val_loss, val_accuracy = 0, 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            val_accuracy += pred.eq(target.view_as(pred)).sum().item()
        val_loss /= len(val_data)
        val_accuracy /= len(val_data)
        global best_val_loss
        global best_val_accuracy
        global best_model
        if val_accuracy > best_val_accuracy:
            print('Saving model...')
            torch.save(model, 'model.pt')
            best_val_loss = val_loss
            best_val_accuracy = val_accuracy
            print('Validation loss: {:.4f}, Validation accuracy: {:.4f}'.format(val_loss, val_accuracy))
            best_model = model
        print('Val loss: {:.4f}, Val accuracy: {:.4f}'.format(val_loss, val_accuracy))
        return best_model, best_val_accuracy

            


In [ ]:
model = DNN()
torch.save(model, "untrained_model.pt")

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

epochs = 10

In [ ]:
for epoch in range(epochs):
    best_model_weights, best_val_acc = train_and_validate(model, criterion, optimizer, train_loader, val_loader)

print(f"Best validation accuracy: {best_val_acc}")

# Model Testing

In [ ]:
# Create a function to test the model on the test data and maintain a list of all the predictions.
def test_model(model, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model.eval()
    test_loss, test_accuracy = 0, 0
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            all_preds.append(pred)
            test_accuracy += pred.eq(target.view_as(pred)).sum().item()
        # test_loss /= len(test_data)
        # test_accuracy /= len(test_data)

        print('Test loss: {:.4f}, Test accuracy: {:.4f}'.format(test_loss, test_accuracy))
    return all_preds
    

In [ ]:
all_preds = test_model(model, val_loader)

In [ ]:
with open("val_preds.txt", "w") as f:
    f.write(str(all_preds))